Python Code Explaination

The purpose of this document is to walk you through the code that my caffeinated brain produced. Please keep in mind that this was the final version of countless versions. While I was transferring my files from my school computer (had to return) to my new computer, all the versions were corrupted, so I salvaged what I could from my physical notebook.

Big Questions: ("A:" was me answering the questions post completion)
- What are the units to use?
    A: Time, number of sips
- How to track hands and not other moving objects in background?
    A: Mediapipe allowed for face and hand tracking. If not present, then there would be no blue/green dots
- What if it's a long sip vs short sip?
    A: By not considering time for when the blue dot is in the detection threshold of the green, it counts 1 very long sip as 1 sip
- How to track in real time?
    A: Introducing the time library lets you keep a time tracker
- How to differentiate desired action (drinking coffee) from undesired action (scratching head)
    A: Creating a threshold (within 3% of camera width) allowed me to do scratch my head or fix my glasses without setting of detector

In [1]:
import cv2
import mediapipe as mp
import time
import math

Imported Libraries
- cv2: need for using compueter camera
- mediapipe: need this library for pre made mapping for face and hands
- time: need for study timer
- math: need for addition and calculating distance threshold for +1 to action or +0

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

mp_holistic is required to map the face and hand
mp_drawing is put in to digitally draw (Note: Probably don't need)

In [ ]:
cap = cv2.VideoCapture(0)
start_time = time.time()
coffee_count = 0
is_sipping = False

Setting up the variables and the objective of the code

In [ ]:
with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Can't grab frame")
            break

Creating the detection thresholds for tracking the finger and the face and making sure that the camera is working

In [ ]:
frame = cv2.flip(frame, 1)
image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
results = holistic.process(image)
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
height, width, _ = image.shape

Conditioning the camera
- Changing view so it's a mirror rather than a Zoom meeting perspective
- Changing colors to be compatible (OpenCV uses BGR; Mediapipe uses RGB)
- Labeling image coordinates for tracking hand and face

In [ ]:
hand_near_mouth = False 

if results.face_landmarks:
    mouth_x = (results.face_landmarks.landmark[13].x +
               results.face_landmarks.landmark[14].x) / 2 * width
    mouth_y = (results.face_landmarks.landmark[13].y +
               results.face_landmarks.landmark[14].y) / 2 * height
    cv2.circle(image, (int(mouth_x), int(mouth_y)), 5, (0, 255, 0), -1)
else:
    mouth_x, mouth_y = None, None

Using pre set conditions to find face and plot a green dot at mouth

In [ ]:
if results.left_hand_landmarks:
    left_index = results.left_hand_landmarks.landmark[12]
    left_x = int(left_index.x * width)
    left_y = int(left_index.y * height)
    cv2.circle(image, (left_x, left_y), 5, (255, 0, 0), -1)
    distance = math.dist([left_x, left_y], [mouth_x, mouth_y])
    if distance < width * 0.03:
        hand_near_mouth = True

Using the pre set conditions to see if the left hand is present and plotting with blue dot.
- used the middle finger (setting 12)

If the blue dot (middle finger) is within 3% of the camera frame's width to the green dot (mouth), then count it

In [ ]:
if results.right_hand_landmarks:
    right_index = results.right_hand_landmarks.landmark[12]
    right_x = int(right_index.x * width)
    right_y = int(right_index.y * height)
    cv2.circle(image, (right_x, right_y), 5, (255, 0, 0), -1)
    distance = math.dist([left_x, left_y], [mouth_x, mouth_y])
    if distance < width * 0.03:
        hand_near_mouth = True

Did the same exact setting for the right hand.

In [ ]:
if hand_near_mouth and not is_sipping:
    coffee_count += 1
    is_sipping = True
elif not hand_near_mouth:
    is_sipping = False

Counting up the times that I sipped the coffee

In [ ]:
elapsed_time = time.time() - start_time

cv2.putText(image, f"Coffee Sip Count: {coffee_count}", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
cv2.putText(image, f"Study Timer: {int(elapsed_time)} sec", (10, 70),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

cv2.imshow('Coffee Drinking Count', image)

Keeps track of the time that passed while displaying the timer and the coffee sip count

In [ ]:
if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()

Exiting out of the model by clicking "q" on keyboard